In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import random
import torch
import torch.nn as nn
from custommodels import LoadDataset, ResNet50, DenseNet121, MobileNetV2
from training import ModelTrainer

In [2]:
# to reproduce
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
db = "HAM"
root = f"../../dataset/CAI 2025/{db}/"
df_train = pd.read_csv(f"{root}/dataframe/df_distance_train.csv")
df_valid = pd.read_csv(f"{root}/dataframe/df_distance_valid.csv")
df_test = pd.read_csv(f"{root}/dataframe/df_distance_test.csv")

loader = LoadDataset(label="label", batch_size=16)
train_loader, valid_loader, test_loader = loader.create_dataloaders(df_train, df_valid, df_test)

Width: 200 Height: 150


# ResNet

In [4]:
models = ["RES", "DENSE", "MOBILE"]

model_save_directory = f"{root}models/{models[0]}/"
if not os.path.exists(model_save_directory):
    os.makedirs(model_save_directory)
    print(f"{model_save_directory} created.")
else:
    print(f"{model_save_directory} aready exist.")

df_valid_filepath = f"{root}eval/df_valid_{models[0]}.csv"
df_test_filepath = f"{root}eval/df_test_{models[0]}.csv"

../../dataset/CAI 2025/HAM/models/RES/ created.


In [ ]:
num_class = 8
num_epochs = 30
lr = 1e-6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50(num_class).to(device)
trainer = ModelTrainer()
best_val_file = trainer.train(model, train_loader, valid_loader, model_save_directory, num_epochs=num_epochs, lr=lr)

/home/kuniko/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Validation Accuracy: 0.6064 | Loss: 1.8714 | F1: 0.1252
Validation Accuracy: 0.6628 | Loss: 1.7407 | F1: 0.1206
Validation Accuracy: 0.6668 | Loss: 1.5768 | F1: 0.1212
Validation Accuracy: 0.6688 | Loss: 1.3955 | F1: 0.1201
Validation Accuracy: 0.6693 | Loss: 1.3218 | F1: 0.1204
Validation Accuracy: 0.6733 | Loss: 1.2890 | F1: 0.1317
Validation Accuracy: 0.6708 | Loss: 1.1821 | F1: 0.1202
Validation Accuracy: 0.6708 | Loss: 1.0986 | F1: 0.1187
Validation Accuracy: 0.6733 | Loss: 1.0903 | F1: 0.1265
Validation Accuracy: 0.6718 | Loss: 1.0479 | F1: 0.1214
Validation Accuracy: 0.6713 | Loss: 1.0474 | F1: 0.1214
Validation Accuracy: 0.6733 | Loss: 1.0271 | F1: 0.1276
Validation Accuracy: 0.6723 | Loss: 0.9918 | F1: 0.1226
Validation Accuracy: 0.6728 | Loss: 0.9926 | F1: 0.1272
Validation Accuracy: 0.6728 | Loss: 0.9906 | F1: 0.1262
Validation Accuracy: 0.6723 | Loss: 0.9566 | F1: 0.1258
Validation Accuracy: 0.6753 | Loss: 0.9315 | F1: 0.1336
Validation Accuracy: 0.6768 | Loss: 0.9411 | F1:

In [ ]:
model = ResNet50(num_class).to(device)
model.load_state_dict(torch.load(best_val_file))

test_predictions, test_outputs = trainer.evaluate(model, valid_loader)
df_valid["pred"] = test_predictions
df_valid["proba"] = test_outputs
df_valid.to_csv(df_valid_filepath, index=False)

In [ ]:
test_predictions, test_outputs = trainer.evaluate(model, test_loader)
df_test["pred"] = test_predictions
df_test["proba"] = test_outputs
df_test.to_csv(df_test_filepath, index=False)